# Tutorial Completo: Monitoraggio della Glicemia Tramite ECG## Panoramica del ProgettoQuesto notebook fornisce un tutorial completo sull'implementazione di un sistema di monitoraggio della glicemia basato su segnali ECG (elettrocardiogramma). L'implementazione segue la metodologia descritta nel paper:**"Glucose level monitoring by ECG"** (Dave et al., 2024)- DOI: https://doi.org/10.1016/j.bspc.2024.106569### Obiettivi del NotebookIn questo notebook imparerai a:1. 🔬 **Caricare e preprocessare** dati ECG e CGM2. 📊 **Visualizzare** segnali ECG e livelli di glucosio3. 🎯 **Estrarre features** morfologiche e HRV dal segnale ECG4. 🤖 **Addestrare modelli** di machine learning per rilevare ipoglicemia/iperglicemia5. 📈 **Valutare le performance** dei modelli6. 🔄 **Eseguire workflow completi** end-to-end### DatasetIl progetto utilizza il dataset **D1NAMO ECG-Glucose**:- URL: https://www.kaggle.com/datasets/sarabhian/d1namo-ecg-glucose-data- ECG: 250 Hz (Zephyr Bioharness)- CGM: Intervalli di 5 minuti (Dexcom CGM)### Modelli Implementati- **MBeat**: Modello Random Forest a livello di battito- **MHRV**: Modello basato su features HRV- **MMorph**: Modello con aggregazione morfologica- **MMorph+HRV**: Modello combinato- **Fusion Model**: Ensemble multi-soglia

## 1. Setup e Installazione### Requisiti di Sistema- Python 3.8+- Librerie specificate in `requirements.txt`### Installazione delle DipendenzePrima di eseguire questo notebook, assicurati di aver installato tutte le dipendenze necessarie.

In [ ]:
# Installazione delle dipendenze (decommentare se necessario)# !pip install -r requirements.txt# Importazione delle librerie principaliimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsfrom pathlib import Pathimport warningswarnings.filterwarnings('ignore')# Configurazione stile graficiplt.style.use('seaborn-v0_8-darkgrid')sns.set_palette("husl")print("✅ Librerie importate con successo!")print(f"NumPy version: {np.__version__}")print(f"Pandas version: {pd.__version__}")

## 2. Importazione Moduli del ProgettoImportiamo tutti i moduli necessari dal nostro progetto.

In [ ]:
# Importazione moduli del progettofrom src.data_loader import SyntheticDataGenerator, D1NAMODataLoaderfrom src.signal_processing import ECGProcessorfrom src.feature_extraction import MorphologyFeatureExtractor, HRVFeatureExtractorfrom src.models import MBeat, MHRV, MMorph, MMorphHRV, FusionModelfrom src.pipeline import GlucoseMonitoringPipeline, run_personalized_trainingprint("✅ Tutti i moduli del progetto sono stati importati con successo!")

## 3. Generazione e Caricamento Dati### 3.1 Dati Sintetici (per Test)Per iniziare senza il dataset completo, possiamo generare dati sintetici che simulano segnali ECG e valori di glucosio.

In [ ]:
# Generazione dati sinteticigenerator = SyntheticDataGenerator(sampling_rate=250)patient_data = generator.generate_patient_data(    duration_hours=2,  # 2 ore di dati    patient_id='synthetic_001')# Informazioni sui dati generatiprint("=" * 60)print("DATI SINTETICI GENERATI")print("=" * 60)print(f"📊 Campioni ECG: {len(patient_data['ecg_signal'])}")print(f"⏱️  Frequenza di campionamento: {patient_data['sampling_rate']} Hz")print(f"🕐 Durata: {len(patient_data['ecg_signal']) / patient_data['sampling_rate'] / 3600:.2f} ore")print(f"🍬 Letture CGM: {len(patient_data['cgm_values'])}")print(f"📈 Range glicemia: {patient_data['cgm_values'].min():.1f} - {patient_data['cgm_values'].max():.1f} mg/dL")print(f"💓 Valori glucosio allineati: {len(patient_data['aligned_glucose'])}")print("=" * 60)

### 3.2 Caricamento Dati Reali (Dataset D1NAMO)Se hai scaricato il dataset D1NAMO, puoi caricarlo con il seguente codice.**Nota**: Decommentare solo se hai il dataset disponibile in `data/raw/`.

In [ ]:
# Caricamento dati reali (decommentare se disponibili)# loader = D1NAMODataLoader(data_dir='data/raw')# # # Ottenere lista pazienti disponibili# patient_ids = loader.get_patient_ids()# print(f"Pazienti disponibili: {patient_ids}")# # # Caricare dati di un paziente specifico# if len(patient_ids) > 0:#     patient_data_real = loader.load_patient_data(patient_ids[0])#     print(f"Dati caricati per paziente: {patient_ids[0]}")# Per ora utilizziamo i dati sinteticiprint("✅ Utilizziamo i dati sintetici per questo tutorial")

## 4. Visualizzazione Dati### 4.1 Visualizzazione Segnale ECG

In [ ]:
# Visualizzazione di 10 secondi di segnale ECGfig, ax = plt.subplots(figsize=(15, 4))# Seleziona 10 secondi di segnalesampling_rate = patient_data['sampling_rate']duration = 10  # secondisamples = duration * sampling_rateecg_segment = patient_data['ecg_signal'][:samples]time_axis = np.arange(len(ecg_segment)) / sampling_rate# Plotax.plot(time_axis, ecg_segment, linewidth=0.8, color='#2E86AB')ax.set_xlabel('Tempo (secondi)', fontsize=12)ax.set_ylabel('Ampiezza ECG (mV)', fontsize=12)ax.set_title('Segnale ECG - Primi 10 Secondi', fontsize=14, fontweight='bold')ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()print(f"✅ Visualizzati {duration} secondi di segnale ECG")

### 4.2 Visualizzazione Valori di Glucosio

In [ ]:
# Visualizzazione valori di glucosio nel tempofig, ax = plt.subplots(figsize=(15, 5))# Creazione asse temporale per CGMcgm_times = patient_data['cgm_times']cgm_values = patient_data['cgm_values']# Plot valori di glucosioax.plot(cgm_times, cgm_values, marker='o', linestyle='-',         linewidth=2, markersize=4, color='#A23B72', label='Glucosio CGM')# Soglie clinicheax.axhline(y=70, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Ipoglicemia (< 70 mg/dL)')ax.axhline(y=180, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Iperglicemia (> 180 mg/dL)')ax.fill_between(cgm_times, 70, 180, alpha=0.1, color='green', label='Range normale')ax.set_xlabel('Tempo (minuti)', fontsize=12)ax.set_ylabel('Glucosio (mg/dL)', fontsize=12)ax.set_title('Andamento del Glucosio nel Tempo', fontsize=14, fontweight='bold')ax.legend(loc='best', fontsize=10)ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()# Statisticheprint(f"📊 Statistiche Glucosio:")print(f"   Media: {cgm_values.mean():.1f} mg/dL")print(f"   Std Dev: {cgm_values.std():.1f} mg/dL")print(f"   Min: {cgm_values.min():.1f} mg/dL")print(f"   Max: {cgm_values.max():.1f} mg/dL")print(f"   Episodi ipoglicemia: {(cgm_values < 70).sum()}")print(f"   Episodi iperglicemia: {(cgm_values > 180).sum()}")

## 5. Elaborazione del Segnale ECG### 5.1 Preprocessing del SegnaleIl preprocessing include:- Filtraggio passa-banda (0.5-40 Hz)- Rimozione del drift della baseline- Normalizzazione del segnale

In [ ]:
# Inizializzazione processore ECGecg_processor = ECGProcessor(sampling_rate=sampling_rate)# Preprocessing del segnaleecg_signal = patient_data['ecg_signal']cleaned_ecg = ecg_processor.preprocess(ecg_signal)print("✅ Segnale ECG preprocessato con successo")# Confronto segnale raw vs preprocessatofig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 8))# Seleziona 5 secondiduration = 5samples = duration * sampling_ratetime_axis = np.arange(samples) / sampling_rate# Segnale rawax1.plot(time_axis, ecg_signal[:samples], linewidth=0.8, color='#2E86AB')ax1.set_ylabel('Ampiezza (mV)', fontsize=11)ax1.set_title('Segnale ECG Originale', fontsize=12, fontweight='bold')ax1.grid(True, alpha=0.3)# Segnale preprocessatoax2.plot(time_axis, cleaned_ecg[:samples], linewidth=0.8, color='#06A77D')ax2.set_xlabel('Tempo (secondi)', fontsize=11)ax2.set_ylabel('Ampiezza (mV)', fontsize=11)ax2.set_title('Segnale ECG Preprocessato', fontsize=12, fontweight='bold')ax2.grid(True, alpha=0.3)plt.tight_layout()plt.show()

### 5.2 Rilevamento Punti FiducialiRileviamo i punti caratteristici dell'ECG (P, Q, R, S, T) su ogni battito.

In [ ]:
# Rilevamento punti fiducialifiducial_points = ecg_processor.detect_fiducial_points(cleaned_ecg)print(f"✅ Punti fiduciali rilevati:")print(f"   Picchi R: {len(fiducial_points['R_peaks'])}")print(f"   Picchi P: {len(fiducial_points.get('P_peaks', []))}")print(f"   Picchi Q: {len(fiducial_points.get('Q_peaks', []))}")print(f"   Picchi S: {len(fiducial_points.get('S_peaks', []))}")print(f"   Picchi T: {len(fiducial_points.get('T_peaks', []))}")# Calcolo frequenza cardiaca mediar_peaks = fiducial_points['R_peaks']if len(r_peaks) > 1:    rr_intervals = np.diff(r_peaks) / sampling_rate  # in secondi    heart_rate = 60 / rr_intervals.mean()    print(f"   💓 Frequenza cardiaca media: {heart_rate:.1f} BPM")

### 5.3 Visualizzazione Battito con Punti Fiduciali

In [ ]:
# Visualizzazione di un singolo battito con punti fiducialiif len(r_peaks) >= 3:    fig, ax = plt.subplots(figsize=(15, 5))        # Seleziona il secondo battito    beat_idx = 1    r_peak = r_peaks[beat_idx]        # Finestra attorno al picco R (± 0.4 secondi)    window = int(0.4 * sampling_rate)    start_idx = max(0, r_peak - window)    end_idx = min(len(cleaned_ecg), r_peak + window)        beat_segment = cleaned_ecg[start_idx:end_idx]    time_axis_beat = (np.arange(len(beat_segment)) - window) / sampling_rate * 1000  # in ms        # Plot battito    ax.plot(time_axis_beat, beat_segment, linewidth=2, color='#2E86AB', label='Battito ECG')        # Marca punti fiduciali    fiducial_types = ['P_peaks', 'Q_peaks', 'R_peaks', 'S_peaks', 'T_peaks']    colors = ['purple', 'blue', 'red', 'green', 'orange']    markers = ['v', 'o', '^', 'o', 'v']        for fid_type, color, marker in zip(fiducial_types, colors, markers):        if fid_type in fiducial_points and len(fiducial_points[fid_type]) > beat_idx:            fid_peak = fiducial_points[fid_type][beat_idx]            if start_idx <= fid_peak < end_idx:                x_pos = (fid_peak - start_idx - window) / sampling_rate * 1000                y_pos = cleaned_ecg[fid_peak]                ax.plot(x_pos, y_pos, marker=marker, markersize=12,                        color=color, label=f'Punto {fid_type[0]}')        ax.set_xlabel('Tempo (ms)', fontsize=12)    ax.set_ylabel('Ampiezza (mV)', fontsize=12)    ax.set_title(f'Battito ECG con Punti Fiduciali (Battito #{beat_idx+1})',                 fontsize=14, fontweight='bold')    ax.legend(loc='best', fontsize=10)    ax.grid(True, alpha=0.3)    plt.tight_layout()    plt.show()else:    print("⚠️  Non abbastanza battiti per la visualizzazione")

## 6. Estrazione Features### 6.1 Features Morfologiche (35 Features per Battito)Le features morfologiche includono:- **Ampiezze** (5): P, Q, R, S, T- **Intervalli** (10): PR, QR, RS, RT, QS, QT, ST, PS, PT, PQ- **Distanze Euclidee** (9): tra punti fiduciali- **Pendenze** (9): tra punti fiduciali- **Addizionali** (2): RR interval, Heart Rate

In [ ]:
# Estrazione features morfologichemorph_extractor = MorphologyFeatureExtractor(sampling_rate=sampling_rate)# Estraiamo features per tutti i battitibeat_features_list = []for beat_idx in range(len(r_peaks)):    beat_features = morph_extractor.extract_beat_features(        cleaned_ecg, fiducial_points, beat_idx    )    if beat_features is not None:        beat_features_list.append(beat_features)# Creazione DataFramebeat_features_df = pd.DataFrame(beat_features_list)print(f"✅ Features morfologiche estratte:")print(f"   Numero di battiti: {len(beat_features_df)}")print(f"   Numero di features per battito: {len(beat_features_df.columns)}")print(f"\n📊 Prime 10 features:")for i, col in enumerate(beat_features_df.columns[:10], 1):    print(f"   {i:2d}. {col}")# Visualizzazione prime righeprint(f"\n📋 Esempio di features (primi 3 battiti):")beat_features_df.head(3)

### 6.2 Features HRV (Heart Rate Variability)Le features HRV vengono calcolate su intervalli temporali e includono:- SDNN, RMSSD, SDSD- NN50, pNN50, NN20, pNN20- HTI, TINN- Mean NN, Median NN, Range NN, IQR- E altre metriche temporali

In [ ]:
# Estrazione features HRVhrv_extractor = HRVFeatureExtractor(sampling_rate=sampling_rate)hrv_features = hrv_extractor.extract_hrv_features(r_peaks)print(f"✅ Features HRV estratte: {len(hrv_features)}")print(f"\n📊 Features HRV disponibili:")# Visualizzazione features HRVif hrv_features:    for i, (key, value) in enumerate(hrv_features.items(), 1):        if isinstance(value, (int, float)):            print(f"   {i:2d}. {key:25s}: {value:10.4f}")        else:            print(f"   {i:2d}. {key:25s}: {value}")else:    print("   ⚠️  Nessuna feature HRV calcolata (potrebbero servire più battiti)")

### 6.3 Visualizzazione Distribuzione Features

In [ ]:
# Visualizzazione distribuzione di alcune features chiavefig, axes = plt.subplots(2, 3, figsize=(15, 8))axes = axes.flatten()# Seleziona alcune features interessantifeatures_to_plot = [    'R_amplitude', 'RR_interval', 'heart_rate',     'PR_interval', 'QT_interval', 'QRS_duration']for idx, feature in enumerate(features_to_plot):    if feature in beat_features_df.columns:        ax = axes[idx]        beat_features_df[feature].hist(bins=30, ax=ax, color='#2E86AB', alpha=0.7, edgecolor='black')        ax.set_title(f'Distribuzione {feature}', fontsize=11, fontweight='bold')        ax.set_xlabel(feature, fontsize=10)        ax.set_ylabel('Frequenza', fontsize=10)        ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()

## 7. Addestramento Modelli### 7.1 Preparazione Dati di TrainingPrepariamo i dati per l'addestramento creando le label (ipoglicemia vs normale) basate sulla soglia di 70 mg/dL.

In [ ]:
# Preparazione dati per trainingthreshold_hypo = 70  # mg/dLaligned_glucose = patient_data['aligned_glucose']# Creazione labels (ipoglicemia = 1, normale = 0)y_labels = (aligned_glucose < threshold_hypo).astype(int)# Rimozione valori NaN dalle featuresX_features = beat_features_df.fillna(beat_features_df.mean()).valuesprint(f"✅ Dati preparati:")print(f"   Campioni totali: {len(X_features)}")print(f"   Features per campione: {X_features.shape[1]}")print(f"   Label ipoglicemia (1): {y_labels.sum()} ({y_labels.sum()/len(y_labels)*100:.1f}%)")print(f"   Label normale (0): {(y_labels == 0).sum()} ({(y_labels == 0).sum()/len(y_labels)*100:.1f}%)")

### 7.2 Split Temporale Train-Test**Importante**: Usiamo uno split temporale (non casuale) per evitare data leakage.- Training set: Prime 75% del tempo- Test set: Ultimi 25% del tempo

In [ ]:
# Split temporalesplit_point = int(len(X_features) * 0.75)X_train = X_features[:split_point]y_train = y_labels[:split_point]X_test = X_features[split_point:]y_test = y_labels[split_point:]print(f"✅ Dataset diviso temporalmente:")print(f"   Training set:")print(f"      - Campioni: {len(X_train)}")print(f"      - Ipoglicemia: {y_train.sum()} ({y_train.sum()/len(y_train)*100:.1f}%)")print(f"   Test set:")print(f"      - Campioni: {len(X_test)}")print(f"      - Ipoglicemia: {y_test.sum()} ({y_test.sum()/len(y_test)*100:.1f}%)")

### 7.3 Addestramento Modello MBeat (Beat-Level)Il modello MBeat è un Random Forest che opera a livello di battito singolo.

In [ ]:
# Addestramento modello MBeatprint("🚀 Addestramento modello MBeat in corso...")mbeat_model = MBeat(n_estimators=100, random_state=42)mbeat_model.train(X_train, y_train)print("✅ Modello MBeat addestrato con successo!")# Predizioni sul test sety_pred = mbeat_model.predict(X_test)y_proba = mbeat_model.predict_proba(X_test)[:, 1]print(f"\n📊 Predizioni completate:")print(f"   Predizioni positive: {y_pred.sum()} ({y_pred.sum()/len(y_pred)*100:.1f}%)")

### 7.4 Valutazione del Modello

In [ ]:
# Valutazione metrichemetrics = mbeat_model.evaluate(X_test, y_test)print("=" * 60)print("METRICHE DI PERFORMANCE - MODELLO MBEAT")print("=" * 60)for metric_name, value in metrics.items():    print(f"{metric_name:20s}: {value:.4f}")print("=" * 60)# Interpretazione metricheprint("\n📖 Interpretazione:")print(f"   • AUC: {metrics['auc']:.4f} - Capacità discriminativa del modello")print(f"   • Sensitivity: {metrics['sensitivity']:.4f} - % ipoglicemie correttamente rilevate")print(f"   • Specificity: {metrics['specificity']:.4f} - % normali correttamente identificati")print(f"   • F1-Score: {metrics['f1_score']:.4f} - Bilanciamento precision-recall")

### 7.5 Matrice di Confusione

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay# Calcolo matrice di confusionecm = confusion_matrix(y_test, y_pred)# Visualizzazionefig, ax = plt.subplots(figsize=(8, 6))disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Normale', 'Ipoglicemia'])disp.plot(cmap='Blues', ax=ax, values_format='d')ax.set_title('Matrice di Confusione - Modello MBeat', fontsize=14, fontweight='bold')plt.tight_layout()plt.show()# Statisticheif cm.size == 4:    tn, fp, fn, tp = cm.ravel()    print(f"\n📊 Dettaglio Predizioni:")    print(f"   True Negatives (TN):  {tn} - Normali correttamente predetti")    print(f"   False Positives (FP): {fp} - Falsi allarmi")    print(f"   False Negatives (FN): {fn} - Ipoglicemie mancate")    print(f"   True Positives (TP):  {tp} - Ipoglicemie correttamente rilevate")

### 7.6 Curva ROC

In [ ]:
from sklearn.metrics import roc_curve, auc# Calcolo curva ROCfpr, tpr, thresholds = roc_curve(y_test, y_proba)roc_auc = auc(fpr, tpr)# Plot curva ROCfig, ax = plt.subplots(figsize=(10, 7))ax.plot(fpr, tpr, color='#2E86AB', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')ax.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random classifier')ax.set_xlim([0.0, 1.0])ax.set_ylim([0.0, 1.05])ax.set_xlabel('False Positive Rate (1 - Specificity)', fontsize=12)ax.set_ylabel('True Positive Rate (Sensitivity)', fontsize=12)ax.set_title('Receiver Operating Characteristic (ROC) Curve', fontsize=14, fontweight='bold')ax.legend(loc='lower right', fontsize=11)ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()

## 8. Pipeline Completa End-to-End### 8.1 Utilizzo della Pipeline AutomatizzataLa funzione `run_personalized_training()` esegue l'intero workflow automaticamente.

In [ ]:
# Pipeline completa automatizzataprint("🚀 Esecuzione pipeline completa...")print("   (Questo può richiedere alcuni minuti)\n")pipeline, metrics_pipeline = run_personalized_training(    patient_data,    task='hypoglycemia',    threshold=70)if metrics_pipeline is not None:    print("\n" + "=" * 60)    print("RISULTATI PIPELINE COMPLETA")    print("=" * 60)    for metric_name, value in metrics_pipeline.items():        print(f"{metric_name:20s}: {value:.4f}")    print("=" * 60)else:    print("⚠️  Pipeline non ha prodotto risultati (dati insufficienti)")

### 8.2 Rilevamento IperglicemiaPossiamo anche addestrare modelli per rilevare l'iperglicemia (glicemia > 180 mg/dL).

In [ ]:
# Training per iperglicemiaprint("🚀 Addestramento modello per iperglicemia...\n")pipeline_hyper, metrics_hyper = run_personalized_training(    patient_data,    task='hyperglycemia',    threshold=180)if metrics_hyper is not None:    print("\n" + "=" * 60)    print("RISULTATI RILEVAMENTO IPERGLICEMIA")    print("=" * 60)    for metric_name, value in metrics_hyper.items():        print(f"{metric_name:20s}: {value:.4f}")    print("=" * 60)else:    print("⚠️  Addestramento iperglicemia non riuscito")

## 9. Importanza delle FeaturesAnalizziamo quali features sono più importanti per il modello.

In [ ]:
# Estrazione importanza featuresif hasattr(mbeat_model.model, 'feature_importances_'):    feature_importance = mbeat_model.model.feature_importances_    feature_names = beat_features_df.columns        # Creazione DataFrame con importanze    importance_df = pd.DataFrame({        'Feature': feature_names,        'Importance': feature_importance    }).sort_values('Importance', ascending=False)        # Plot top 15 features più importanti    fig, ax = plt.subplots(figsize=(12, 8))    top_n = 15    top_features = importance_df.head(top_n)        ax.barh(range(top_n), top_features['Importance'].values, color='#2E86AB', alpha=0.8)    ax.set_yticks(range(top_n))    ax.set_yticklabels(top_features['Feature'].values)    ax.set_xlabel('Importanza', fontsize=12)    ax.set_title(f'Top {top_n} Features Più Importanti', fontsize=14, fontweight='bold')    ax.invert_yaxis()    ax.grid(True, alpha=0.3, axis='x')    plt.tight_layout()    plt.show()        print(f"\n📊 Top 10 Features per Importanza:")    for i, row in importance_df.head(10).iterrows():        print(f"   {row['Feature']:25s}: {row['Importance']:.4f}")else:    print("⚠️  Importanza features non disponibile per questo modello")

## 10. Utilizzo Avanzato### 10.1 Confronto Modelli MultipliConfrontiamo le performance di diversi modelli.

In [ ]:
# Confronto modelli (richiede più tempo)print("🔬 Confronto di modelli multipli...")print("   (Questo può richiedere diversi minuti)\n")models_comparison = {}# MBeat (già addestrato)models_comparison['MBeat'] = metrics# Puoi aggiungere altri modelli qui# Ad esempio: MHRV, MMorph+HRV, ecc.# Visualizzazione confrontoif len(models_comparison) > 0:    comparison_df = pd.DataFrame(models_comparison).T        print("=" * 70)    print("CONFRONTO MODELLI")    print("=" * 70)    print(comparison_df.to_string())    print("=" * 70)        # Plot confronto metriche    fig, ax = plt.subplots(figsize=(12, 6))    comparison_df[['auc', 'sensitivity', 'specificity', 'f1_score']].plot(        kind='bar', ax=ax, rot=0, width=0.8    )    ax.set_ylabel('Score', fontsize=12)    ax.set_title('Confronto Performance tra Modelli', fontsize=14, fontweight='bold')    ax.legend(loc='best', fontsize=10)    ax.grid(True, alpha=0.3, axis='y')    ax.set_ylim([0, 1.0])    plt.tight_layout()    plt.show()

### 10.2 Salvataggio e Caricamento Modelli

In [ ]:
import os# Creazione directory per i modellios.makedirs('models', exist_ok=True)# Salvataggio modellomodel_path = 'models/mbeat_tutorial.pkl'mbeat_model.save(model_path)print(f"✅ Modello salvato in: {model_path}")# Caricamento modelloloaded_model = MBeat()loaded_model.load(model_path)print(f"✅ Modello caricato da: {model_path}")# Verifica che il modello caricato funzionitest_pred = loaded_model.predict(X_test[:5])print(f"\n🧪 Test predizione con modello caricato: {test_pred}")

## 11. Predizione in Tempo Reale (Simulata)Simuliamo come il modello potrebbe essere usato per predizioni in tempo reale.

In [ ]:
# Simulazione predizione in tempo realeprint("🎯 Simulazione Predizioni in Tempo Reale")print("=" * 60)# Prendiamo gli ultimi 10 battiti del test setrecent_beats = X_test[-10:]recent_glucose = aligned_glucose[split_point:][-10:]# Predizionipredictions = mbeat_model.predict(recent_beats)probabilities = mbeat_model.predict_proba(recent_beats)[:, 1]print(f"\n{'Battito':<10} {'Glucosio':<12} {'Predizione':<15} {'Probabilità':<15} {'Status'}")print("-" * 70)for i, (gluc, pred, prob) in enumerate(zip(recent_glucose, predictions, probabilities)):    status = "⚠️  ALLARME" if pred == 1 else "✅ OK"    pred_text = "Ipoglicemia" if pred == 1 else "Normale"    print(f"{i+1:<10} {gluc:<12.1f} {pred_text:<15} {prob:<15.3f} {status}")print("=" * 60)

## 12. Consigli Pratici e Troubleshooting### 12.1 Best Practices1. **Split Temporale**: Usa sempre split temporali (non casuali) per evitare data leakage2. **Normalizzazione**: Considera la normalizzazione delle features per modelli diversi da Random Forest3. **Gestione Class Imbalance**: Valuta tecniche di bilanciamento se le classi sono molto sbilanciate4. **Validazione**: Usa cross-validation temporale con multiple finestre di test5. **Feature Engineering**: Sperimenta con nuove features o combinazioni### 12.2 Problemi Comuni**Problema**: Features estratte insufficienti- **Soluzione**: Assicurati di avere almeno 1-2 minuti di segnale ECG continuo**Problema**: Metriche molto basse- **Soluzione**: Verifica la qualità del segnale ECG e la presenza di eventi di ipoglicemia nel dataset**Problema**: Errori di memoria- **Soluzione**: Elabora i dati in chunk più piccoli### 12.3 Estensioni Possibili- Implementare modelli HRV puri (MHRV)- Creare ensemble di modelli (Fusion Model)- Aggiungere features nel dominio della frequenza- Implementare deep learning models (LSTM, CNN)- Creare dashboard interattiva con Streamlit

## 13. Esempio Workflow CompletoRicapitoliamo l'intero processo in un esempio completo e autocontenuto.

In [ ]:
def complete_workflow_example():    """    Esempio di workflow completo per il monitoraggio della glicemia tramite ECG    """    print("🔄 WORKFLOW COMPLETO: Monitoraggio Glicemia da ECG")    print("=" * 70)        # 1. Generazione dati    print("\n1️⃣  Generazione dati sintetici...")    generator = SyntheticDataGenerator(sampling_rate=250)    data = generator.generate_patient_data(duration_hours=2)    print(f"   ✅ Generati {len(data['ecg_signal'])} campioni ECG")        # 2. Preprocessing    print("\n2️⃣  Preprocessing segnale ECG...")    processor = ECGProcessor(sampling_rate=250)    cleaned = processor.preprocess(data['ecg_signal'])    fiducials = processor.detect_fiducial_points(cleaned)    print(f"   ✅ Rilevati {len(fiducials['R_peaks'])} battiti")        # 3. Feature extraction    print("\n3️⃣  Estrazione features...")    morph_ext = MorphologyFeatureExtractor(sampling_rate=250)    features = []    for i in range(len(fiducials['R_peaks'])):        feat = morph_ext.extract_beat_features(cleaned, fiducials, i)        if feat is not None:            features.append(feat)    features_df = pd.DataFrame(features)    print(f"   ✅ Estratte {len(features_df.columns)} features per {len(features_df)} battiti")        # 4. Preparazione dati    print("\n4️⃣  Preparazione dati di training...")    X = features_df.fillna(features_df.mean()).values    y = (data['aligned_glucose'] < 70).astype(int)    split = int(len(X) * 0.75)    X_tr, X_te = X[:split], X[split:]    y_tr, y_te = y[:split], y[split:]    print(f"   ✅ Train: {len(X_tr)} campioni, Test: {len(X_te)} campioni")        # 5. Training    print("\n5️⃣  Addestramento modello...")    model = MBeat(n_estimators=100, random_state=42)    model.train(X_tr, y_tr)    print(f"   ✅ Modello addestrato")        # 6. Valutazione    print("\n6️⃣  Valutazione performance...")    metrics = model.evaluate(X_te, y_te)    print(f"   ✅ AUC: {metrics['auc']:.4f}")    print(f"   ✅ Sensitivity: {metrics['sensitivity']:.4f}")    print(f"   ✅ Specificity: {metrics['specificity']:.4f}")    print(f"   ✅ F1-Score: {metrics['f1_score']:.4f}")        print("\n" + "=" * 70)    print("✅ WORKFLOW COMPLETATO CON SUCCESSO!")    print("=" * 70)        return model, metrics# Esecuzione workflowtry:    trained_model, final_metrics = complete_workflow_example()except Exception as e:    print(f"⚠️  Errore durante il workflow: {e}")

## 14. Risorse e Riferimenti### Riferimenti Scientifici1. **Paper Principale**:   - Dave, D., et al. (2024). "Glucose level monitoring by ECG"   - *Biomedical Signal Processing and Control*, Volume 96   - DOI: https://doi.org/10.1016/j.bspc.2024.1065692. **Dataset D1NAMO**:   - https://www.kaggle.com/datasets/sarabhian/d1namo-ecg-glucose-data### Documentazione Librerie- **NumPy**: https://numpy.org/doc/- **Pandas**: https://pandas.pydata.org/docs/- **Scikit-learn**: https://scikit-learn.org/stable/- **NeuroKit2**: https://neuropsychology.github.io/NeuroKit/- **Matplotlib**: https://matplotlib.org/stable/contents.html### Codice Sorgente- **Repository GitHub**: Tutti i file sorgente sono disponibili nella directory `src/`  - `signal_processing.py`: Preprocessing ECG e rilevamento fiduciali  - `feature_extraction.py`: Estrazione features morfologiche e HRV  - `models.py`: Implementazione modelli ML  - `pipeline.py`: Pipeline end-to-end  - `data_loader.py`: Caricamento dati### File di Esempio- `example.py`: Script di esempio con dati sintetici- `train_all_patients.py`: Script per training su tutti i pazienti- `USAGE_GUIDE.md`: Guida dettagliata all'utilizzo### Contatti e SupportoPer domande, problemi o contributi:- Apri una issue sul repository GitHub- Consulta il `README.md` per informazioni aggiuntive

## 15. Riepilogo### Cosa Abbiamo ImparatoIn questo notebook abbiamo coperto:✅ **Caricamento Dati**: Come caricare e generare dati ECG e CGM✅ **Preprocessing**: Filtraggio e pulizia del segnale ECG✅ **Fiducial Detection**: Rilevamento automatico di punti P, Q, R, S, T✅ **Feature Extraction**: Estrazione di 35 features morfologiche e 18 HRV✅ **Model Training**: Addestramento di modelli Random Forest✅ **Evaluation**: Calcolo di metriche (AUC, Sensitivity, Specificity, F1)✅ **Visualization**: Grafici di segnali, features, e performance✅ **Complete Pipeline**: Workflow end-to-end automatizzato### Prossimi Passi1. **Sperimenta con Dati Reali**: Scarica il dataset D1NAMO e prova con dati reali2. **Ottimizza Parametri**: Testa diversi iperparametri del modello3. **Prova Altri Modelli**: Implementa MHRV, MMorph+HRV, Fusion Model4. **Feature Engineering**: Crea nuove features personalizzate5. **Deploy**: Crea un'applicazione per uso pratico### Note FinaliQuesto notebook fornisce una base solida per comprendere e implementare sistemi di monitoraggio della glicemia basati su ECG. Ricorda che i risultati possono variare significativamente tra pazienti diversi (personalizzazione è fondamentale) e che la qualità del segnale ECG è critica per ottenere buone performance.**Buona sperimentazione! 🚀**